# GENERATE A BACKEND PROJECT 

+ this notebook is a template to generate Backend-required files to view a project with the DataDiVR (preview or VR)
+ the "create a graph" section is a template graph writing a required format (json) to then use the generate-project functions of the DataDiVR backend 
+ If you have multiple graphs with stored positions, colors etc which should go into one project, use the "merge_graphs" function below 
+ This will result in a dictionary which can be used with the 

# FUNCTIONS 

In [1]:
def merge_graphs(graphs):
    all_nodes = []
    all_links = []
    layouts = []

    seen_nodes = set()  # To track seen node IDs
    seen_links = set()  # To track seen links by a tuple of (source, target)

    for graph in graphs:
        # Process nodes for global and layout-specific lists
        for node, attrs in graph.nodes(data=True):
            if node not in seen_nodes:
                annotation = attrs.get('annotation', [])
                anntation_mod = {}
                for k,v in annotation.items():
                    anntation_mod[k] = [v]
                all_nodes.append({'id': node, 'name': node, 
                                    'annotation': anntation_mod})
                seen_nodes.add(node)

        # Process links for global list, now with separate source and target
        for ix,(source, target, attrs) in enumerate(graph.edges(data=True)):
            if (source, target) not in seen_links:
                # Assuming 'weight' might be an attribute you're interested in
                # weight = attrs.get('weight', 1)  # Example, uncomment if needed
                all_links.append({
                    'id': ix,  # It's unclear how 'id' is determined; might need a unique strategy
                    # 'w': weight,  # Uncomment if using weight
                    'source': source,
                    'target': target
                })
                seen_links.add((source, target))

        # Prepare layout-specific nodes and links
        layout_nodes = [{'nodecolor': attrs.get('nodecolor', ''),
                         'pos': attrs.get('pos', []),
                         'cluster': attrs.get('cluster',''),
                         'id': node} for node, attrs in graph.nodes(data=True)
                        ]
        layout_links = [{'linkcolor': attrs.get('linkcolor', ''),
                         'source': source,
                         'target': target} for source, target, attrs in graph.edges(data=True)]

        layouts.append({'nodes': layout_nodes, 'links': layout_links})

    # Assuming the structure of the graphs are similar, and using the first graph as the base
    graphlayouts = [graph.name for graph in graphs]
    merged_structure = {
        'directed': graphs[0].is_directed(),
        'multigraph': graphs[0].is_multigraph(),
        'graphtitle': graphs[0].graph.get("graphtitle", ""),
        'graphdesc': graphs[0].graph.get("graphdesc", ""),
        'graphlayouts': graphlayouts,
        'annotationTypes': True,
        'nodes': all_nodes,
        'links': all_links,
        'layouts': layouts
    }

    return merged_structure

# STEP 1 - Create Graphs with stored positions, colors etc. or use your own nx.Graphs

### nx.Graph

In [2]:
import networkx as nx
import json 
import os

G = nx.random_geometric_graph(10,1)
print("Number of nodes: ", len(G.nodes()))
print("Number of Links: ", len(G.edges()))

# ===============================================
# GRAPH NAME AND DESCRIPTION - a string each
# ===============================================
G.name = "_uploadtest-5"
G.graph['graphtitle'] = G.name
G.graph['graphdesc'] = "A toy graph for testing purposes. Number of nodes: "+str(len(G.nodes()))+", Links: "+ str(len(G.edges()))+"."

Number of nodes:  10
Number of Links:  42


### create node anntotations

In [3]:
# ===============================================
# ANNOTATIONS FORMAT - new ! (dictionary)
# ===============================================

l_annotations_json = []
d_degree = dict(G.degree())
d_eigen = dict(nx.eigenvector_centrality(G))
for g in G.nodes():
    sublist = {"Node":g, "Degree":d_degree[g], "Eigenv": round(d_eigen[g],2)}
    l_annotations_json.append(sublist)
        
d_annotations = dict(zip(G.nodes(), l_annotations_json))
nx.set_node_attributes(G, d_annotations, name="annotation")


### create node positions and set as "pos" Graph attribute 
here are 3 different layouts, which all are stored in unique nx.Graph-objects (G_rgba, G_hex, ....)

In [4]:
# First layout (i.e. Graph 1)
G_rgba = G.copy()
G_rgba.name = 'layout1-spring'
posG3D_1_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=100)
posG3D_1 = {key: value.tolist() for key, value in posG3D_1_pre.items()}
nx.set_node_attributes(G_rgba, posG3D_1, name="pos")


# Second layout (i.e. Graph 2)
G_hex = G.copy()
G_hex.name = 'layout2-spring'
posG3D_2_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=200)
posG3D_2= {key: value.tolist() for key, value in posG3D_2_pre.items()}
nx.set_node_attributes(G_hex, posG3D_2, name="pos")


# Third layout (i.e. Graph 3)
G_hex8 = G.copy()
G_hex8.name = 'layout3-spring'
posG3D_3_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=500)
posG3D_3 = {key: value.tolist() for key, value in posG3D_3_pre.items()}
nx.set_node_attributes(G_hex8, posG3D_3, name="pos")


# Fourth layout (i.e. Graph 4) - with clusters
G_clusters = G.copy()
G_clusters.name = 'layout4-clusters'
clustername_1 = 'cluster group 1'
clustername_2 = 'cluster group 2'
clustername_3 = 'cluster group 3'

# nodes into groups
for g in G_clusters.nodes():
    if g < len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_1
    elif g < 2 * len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_2
    else:
        G_clusters.nodes[g]['cluster'] = clustername_3

nx.set_node_attributes(G_clusters, posG3D_2, name="pos")

In [5]:
#get node attributes 
#node_attributes = nx.get_node_attributes(G_clusters, 'pos')
#print(node_attributes)

#### node and link colors 

In [6]:
# 3 Formats of colors values are supported: hex, rgba, hex8

# FIRST GRAPH - rgba color values
d_nodecolors_rgba = dict(zip(G_rgba.nodes(),[(255,35,0,120)]*len(G_rgba.nodes())))
nx.set_node_attributes(G_rgba, d_nodecolors_rgba, name="nodecolor")
l_linkcolors_rgba = (0,255,0,100)
nx.set_edge_attributes(G_rgba, l_linkcolors_rgba, name="linkcolor")


# SECOND GRAPH - hex color values 
d_nodecolors_hex = dict(zip(G_hex.nodes(),['#FF2300']*len(G_hex.nodes())))
nx.set_node_attributes(G_hex, d_nodecolors_hex, name="nodecolor")
l_linkcolors_hex = '#ff0000'
nx.set_edge_attributes(G_hex, l_linkcolors_hex, name="linkcolor")


# THIRD GRAPH - hex8 color values
d_nodecolors_hex8 = dict(zip(G_hex8.nodes(),['#0000ffaa']*len(G_hex8.nodes())))
nx.set_node_attributes(G_hex8, d_nodecolors_hex8, name="nodecolor")
l_linkcolors_hex8 = '#0080ffaa'
nx.set_edge_attributes(G_hex8, l_linkcolors_hex8, name="linkcolor")


# FOURTH GRAPH - clusters assigned 

# node colors 
d_nodecolors_clusters = {}
nodes_group1 = []
nodes_group2 = []
nodes_group3 = []
for n in G_clusters.nodes(): 
    if G_clusters.nodes[n]['cluster'] == clustername_1:
        d_nodecolors_clusters[n] = '#0000ffaa'
        nodes_group1.append(n)
    elif G_clusters.nodes[n]['cluster'] == clustername_2:
        d_nodecolors_clusters[n] = '#00ff00aa'
        nodes_group2.append(n)
    elif G_clusters.nodes[n]['cluster'] == clustername_3:
        d_nodecolors_clusters[n] = '#ff0000aa'
        nodes_group3.append(n)

# link colors
d_linkcolors_clusters = {}
for edge in G_clusters.edges():
    if edge[0] in nodes_group1 and edge[1] in nodes_group1:
        d_linkcolors_clusters[edge] = '#0000ff'
       
    elif edge[0] in nodes_group2 and edge[1] in nodes_group2:
        d_linkcolors_clusters[edge] = '#00ff00'
       
    elif edge[0] in nodes_group3 and edge[1] in nodes_group3:
        d_linkcolors_clusters[edge] = '#ff0000'
       
    else:
        d_linkcolors_clusters[edge] = '#B1B1B1'

l_linkcolors_clusters = list(d_linkcolors_clusters.values())

nx.set_node_attributes(G_clusters, d_nodecolors_clusters, name="nodecolor")
nx.set_edge_attributes(G_clusters, {edge: color for edge, color in zip(G_clusters.edges(), l_linkcolors_clusters)}, "linkcolor")


# STEP 2 - Create one file with all graphs using "merge_graphs" 
in the required DataDIVR format

In [7]:
Graphs = [G_rgba, G_hex, G_clusters, G_hex8]
G_merged = merge_graphs(Graphs)

#### (optional) Or take old JSON files and merge into one as input

In [8]:
# import json
# import glob

# def transform_annotation(annotation_list):
#     """
#     Transforms a list containing a single string of annotations into a dictionary.
#     The string is expected to contain annotations separated by '/',
#     with each annotation in the format 'key: value'.
#     Example input: ['ID: 1/degree: 10']
#     Example output: {'ID': '1', 'degree': '10'}
#     """
#     annotation_dict = {}
#     if annotation_list:  # Check if the list is not empty
#         annotations = annotation_list[0].split('/')
#         for annotation in annotations:
#             key, value = annotation.split(': ', 1)  # Split on the first occurrence of ': '
#             annotation_dict[key.strip()] = value.strip()
#     return annotation_dict

# def merge_old_json_structure(file_paths):
#     all_nodes = []
#     all_links = []
#     layouts = []

#     for file_path in file_paths:
#         with open(file_path, 'r') as file:
#             data = json.load(file)

#             # Process nodes for global and layout-specific lists
#             for node in data["nodes"]:
#                 if "annotation" in node and isinstance(node["annotation"], list):
#                     node["annotation"] = transform_annotation(node["annotation"])
#                 all_nodes.append({'id': node['id'], 'annotation': node.get('annotation', {})})

#             # Process links for global list, now with separate source and target
#             for link in data["links"]:
#                 all_links.append({
#                     'source': link['source'],
#                     'target': link['target']
#                 })

#             # Prepare layout-specific nodes and links
#             layout_nodes = [{'nodecolor': node.get('nodecolor', ''),
#                              'pos': node.get('pos', []),
#                              'id': node['id']} for node in data["nodes"]]
#             layout_links = [{'linkcolor': link.get('linkcolor', ''),
#                              'source': link['source'],
#                              'target': link['target']} for link in data["links"]]

#             layouts.append({'nodes': layout_nodes, 'links': layout_links})

#     # Use the structure of the first file as the base for the merged structure
#     merged_structure = {}
#     if file_paths:
#         with open(file_paths[0], 'r') as file:
#             base_structure = json.load(file)
#             # Extract graph properties directly to the top level, not nested under 'graph'
#             graph_info = base_structure.get('graph', {})
#             for key, value in graph_info.items():
#                 merged_structure[key] = value

#             # Set other top-level keys
#             merged_structure['directed'] = base_structure.get('directed', False)
#             merged_structure['multigraph'] = base_structure.get('multigraph', False)

#     # Add the global nodes and links, and the layouts to the merged structure
#     merged_structure['nodes'] = all_nodes
#     merged_structure['links'] = all_links
#     merged_structure['layouts'] = layouts

#     return merged_structure


# # old project ( old JSON format) 
# folder = "temp_upload_data/"
# subfolder = "cube"
# file_paths = glob.glob(folder + subfolder + '/' + '*.json')
# merged_json = merge_old_json_structure(file_paths)

# # Save the merged JSON to a file
# with open(folder + 'merged.json', 'w') as outfile:
#     json.dump(merged_json, outfile, indent=4)

# print("Merged JSON saved as 'merged.json'.")

# merged_json

# STEP 3 - CREATE PROJECT - for DataDiVR BACKEND

In [9]:
from uploaderGraph import upload_filesJSON

upload_filesJSON(G_merged)

Successfully created the directory static/projects/_uploadtest-4-sceneskey 
CDEBUG: created folder and pfile.json
C_DEBUG: pfile :  {'name': '_uploadtest-4-sceneskey', 'layouts': [], 'layoutsRGB': [], 'links': [], 'linksRGB': [], 'selections': [], 'scenes': []}
['_uploadtest-4-sceneskey', 'JSON_barbellgraph', 'JSON_autocore', '_uploadtest-2', 'JSON_Zachary', '_uploadtest-3', 'ToyNetwork_2', 'ToyNetwork_geometric-new', '_uploadtest-1', 'ToyNetwork_1']


'<a style="color:green;">SUCCESS </a>LayoutID0 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID2 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID3 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID0 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID2 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID3 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID0 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID2 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID3 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID0_links Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1_links Link Textures Crea